In [1]:
import pandas as pd
import numpy as np
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from gensim.models.doc2vec import Doc2Vec

from skbio import DistanceMatrix
from skbio.tree import nj
#run: pip install scikit-bio

import uts

Train Data

In [2]:
df = pd.read_csv('data/Form relationships.csv')
gdf = uts.process_data(df)

/home/sunny/miniconda3/envs/env5/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [3]:
#gdf.head(10)
#gdf.get_group('UserProfile')
#gdf.get_group('UserProfileEvent')

Calculate Intersection Matrix

In [12]:
# option_tb 2 : keep table name multiple times
# option_tb 1 : keep table name once
# option_tb 0 : drop table name

# option_p 1 : keep ordinal position
# option_p 0 : drop ordinal position

lst_set = uts.df_set(gdf, 0, 0)

lst_set

[{'Agent Action Id',
  'Description',
  'ML Data Type Id',
  'Micro Service Agent Id',
  'Micro Service Implementation',
  'Name'},
 {'Action',
  'Action Date',
  'Action User Profile Id',
  'Agent Action Audit Id',
  'Agent Action Event Id',
  'Agent Action Id',
  'Field',
  'Field Name',
  'New Value',
  'Previous Value'},
 {'Agent Action Event Id',
  'Agent Action Id',
  'Event Date Time',
  'Event Payload',
  'Event Type',
  'Event User Profile Id',
  'Triggered By Id',
  'Triggered By_ Agent Action Event Id'},
 {'Agent Action Id',
  'Agent Action Implementation Id',
  'Agent Id',
  'Agent Version Id',
  'Name',
  'Python Source'},
 {'Action',
  'Action Date',
  'Action User Profile Id',
  'Agent Action Implementation Audit Id',
  'Agent Action Implementation Event Id',
  'Agent Action Implementation Id',
  'Field',
  'Field Name',
  'New Value',
  'Previous Value'},
 {'Agent Action Implementation Event Id',
  'Agent Action Implementation Id',
  'Event Date Time',
  'Event Payload'

In [5]:
# create matrix
matrix = uts.create_inetrsection_matrix(lst_set)

tables = pd.DataFrame(gdf).loc[:,0]
matrix.set_axis(tables, axis=1, inplace=True)
matrix.rename(tables, inplace=True)
matrix.head()

# matrix to csv
matrix.to_csv('result/result_matrix_intersection.csv')
        

Matrix to tree

In [6]:
dm = DistanceMatrix(matrix, tables)
tree = nj(dm)
str_tree = tree.ascii_art()

text_file = open("result/tree_train_intersection.txt", "w")
text_file.write(str_tree)
text_file.close()

Test Data

In [7]:
ddf = pd.read_csv('data/Form relationships_dummy tables.csv')
gdf_test = uts.process_data(ddf)
#gdf_test.get_group('A')

test_tables = pd.DataFrame(gdf_test).loc[:,0]

# Making table name reference
dummy_table = {}

for i, val in enumerate(test_tables):
    dummy_table[val] = i

dummy_table['A']

0

Calculate Intersection Matrix for Test Data

In [8]:
lst_set_t = uts.df_set(gdf_test, 0, 0)

# create matrix
matrix_t = uts.create_inetrsection_matrix(lst_set_t)

tables_t = pd.DataFrame(gdf_test).loc[:,0]
matrix_t.set_axis(tables_t, axis=1, inplace=True)
matrix_t.rename(tables_t, inplace=True)

# matrix to csv
matrix_t.to_csv('result/result_matrix_intersection_test.csv')
        

In [9]:
matrix_t


,A,A+B,A1,A2,A3,A4,B,B1,C,D
A,0.000000,0.230769,0.000000,0.000000,0.100000,0.000000,0.461538,0.461538,1.0,0.300000
A+B,0.230769,0.000000,0.230769,0.230769,0.307692,0.230769,0.230769,0.230769,1.0,0.461538
A1,0.000000,0.230769,0.000000,0.000000,0.100000,0.000000,0.461538,0.461538,1.0,0.300000
A2,0.000000,0.230769,0.000000,0.000000,0.100000,0.000000,0.461538,0.461538,1.0,0.300000
A3,0.100000,0.307692,0.100000,0.100000,0.000000,0.100000,0.416667,0.416667,1.0,0.222222
A4,0.000000,0.230769,0.000000,0.000000,0.100000,0.000000,0.461538,0.461538,1.0,0.300000
B,0.461538,0.230769,0.461538,0.461538,0.416667,0.461538,0.000000,0.000000,1.0,0.300000
B1,0.461538,0.230769,0.461538,0.461538,0.416667,0.461538,0.000000,0.000000,1.0,0.300000
C,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000
D,0.300000,0.461538,0.300000,0.300000,0.222222,0.300000,0.300000,0.300000,1.0,0.000000


In [10]:
tables_t

0      A
1    A+B
2     A1
3     A2
4     A3
5     A4
6      B
7     B1
8      C
9      D
Name: 0, dtype: object

Distance Tree for Test Data

In [11]:
dm_1 = DistanceMatrix(matrix_t, tables_t)
tree_1 = nj(dm_1)
str_tree = tree_1.ascii_art()

text_file = open("result/tree_test_intersection.txt", "w")
text_file.write(str_tree)
text_file.close()